<a href="https://colab.research.google.com/github/audreyemmely/comp-evolucionaria/blob/main/tsp_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **O problema**

Nesse notebook, usaremos um AG para encontrar uma solução para o problema do caixeiro viajante (travelling salesman problem, TSP). O TSP é descrito da seguinte forma:

*“Dada uma lista de cidades e as distâncias entre cada par de cidades, qual é o trajeto mais curto possível que visita cada cidade e retorna à cidade de origem?”*

![image.png](https://miro.medium.com/max/512/1*DwB6K0KKNug7-y59Uowhsg.png)

Com isso, há duas regras importantes para manter em mente:
1.   Cada cidade precisa ser visitada exatamente uma vez
2.   Devemos retornar à cidade de partida, então nossa distância total precisa ser calculada de acordo

# **A abordagem**



*   Gene: uma cidade (representada como coordenadas (x, y))
*   Indivíduo (também conhecido como "cromossomo"): uma única rota que satisfaça as condições acima
*   População: uma coleção de rotas possíveis (ou seja, coleção de indivíduos)
*   Pais: duas rotas que são combinadas para criar uma nova rota
*   Pool de acasalamento (mating): uma coleção de pais que são usados ​​para criar nossa próxima população (criando assim a próxima geração de rotas)
*   Fitness: uma função que nos diz o quão bom é cada trajeto (no nosso caso, quão curta é a distância)
*   Mutação: uma forma de introduzir variação em nossa população trocando aleatoriamente duas cidades em uma rota
*   Elitismo: uma forma de transportar os melhores indivíduos para a próxima geração

Nosso AG prosseguirá nas seguintes etapas:


1.   Crie a população
2.   Determine o fitness
3.   Selecione o pool de acasalamento
4.   Faça o cruzamento
5.   Faça a mutação
6.   Repita



# **Construindo o algoritmo genético**

Primeiro vamos importar as bibliotecas que serão utilizadas

In [1]:
import numpy as np
import pandas as pd
import random
import operator
import matplotlib.pyplot as plt

### Criando as classes "Cidade" e "Fitness"

A classe ```cidade``` nos permitirá criar e administrar as cidades. Estas são simplesmente nossas coordenadas (x, y). 

Dentro da classe ```cidade```, criamos a função ```calc_distanc```, para calcular a distância usando o teorema de Pitágoras. E também criamos a função  ```__repr__``` para termos uma maneira mais limpa de gerar as cidades como coordenadas.

In [2]:
class Cidade:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def calc_distanc(self, cidade):
        xDis = abs(self.x - cidade.x)
        yDis = abs(self.y - cidade.y)
        calc_distanc = np.sqrt((xDis ** 2) + (yDis ** 2))
        return calc_distanc
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

No nosso caso, a classe ```fitness``` será o inverso da distância da rota. Queremos minimizar a distância da rota, então uma pontuação de fitness maior é melhor. Com base na regra nº 2, precisamos começar e terminar no mesmo lugar, portanto, esse cálculo extra é contabilizado em 
```
if i + 1 < len(self.rota):
      cidadeDeChegada = self.rota[i + 1]
```.

In [3]:
class Fitness:
    def __init__(self, rota):
        self.rota = rota
        self.distancia = 0
        self.fitness = 0.0
    
    def rotaDistancia(self):
        if self.distancia ==0:
            caminhoDistancia = 0
            for i in range(0, len(self.rota)):
                cidadeDePartida = self.rota[i]
                cidadeDeChegada = None
                if i + 1 < len(self.rota):
                    cidadeDeChegada = self.rota[i + 1]
                else:
                    cidadeDeChegada = self.rota[0]
                caminhoDistancia += cidadeDePartida.distancia(cidadeDeChegada)
            self.distancia = caminhoDistancia
        return self.distancia
    
    def rotaFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.rotaDistancia())
        return self.fitness

### Criando a população


Agora vamos criar nossa população inicial (também conhecida como primeira geração). Para fazer isso, precisamos criar uma função que produza rotas que satisfaçam nossas condições (obs.: vamos criar a lista de cidades quando executarmos o AG posteriormente). Para criar um indivíduo, selecionamos aleatoriamente a ordem em que visitamos cada cidade.

In [4]:
def criarRota(listaDeCidades):
  rota = random.sample(listaDeCidades, len(listaDeCidades))
  return rota

Isso produz um indivíduo, mas queremos uma população completa, então vamos fazer isso na próxima função. Iremos percorrer a função ```criarRota``` até que tenhamos quantas rotas quisermos para nossa população.

In [5]:
def populacaoInicial(tamPop, listaDeCidades):
    populacao = []

    for i in range(0, tamPop):
        populacao.append(criarRota(listaDeCidades))
    return populacao